In [ ]:
!pip install mglearn
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mglearn
# 음수 표현
plt.rcParams['axes.unicode_minus'] = False
# 경고 무시
import warnings
warnings.filterwarnings('ignore')
# 매직명령어 : 시각화 결과가 포함되도록
%matplotlib inline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 4.7 MB/s eta 0:00:00


# 1. 데이팅 앱
- 매칭 성사 여부를 예측하기
- 스피드 데이팅은 남녀 수십쌍이 짧은 시간을 보낸 뒤 서로에 대한 호감도를 표현하여 짝을 매칭하는 이벤트
- 독립변수로는 상대방과 내정보, 개인취향, 상대방에 대한 평가 등이 있고 target은 매칭 성사 여부
- XGBoost 모델을 이용하기

In [ ]:
df = pd.read_csv('speed_dating.csv')

In [ ]:
# 코드 추가 (info 위)
pd.options.display.max_columns = 40   # 기본 컬럼 20개에서 40개로 늘리기/ row로 행 조절도 가능

In [ ]:
df.head()

,Unnamed: 0,has_null,gender,age,age_o,race,race_o,importance_same_race,importance_same_religion,pref_o_attractive,pref_o_sincere,pref_o_intelligence,pref_o_funny,pref_o_ambitious,pref_o_shared_interests,attractive_o,sincere_o,intelligence_o,funny_o,ambitous_o,shared_interests_o,attractive_important,sincere_important,intellicence_important,funny_important,ambtition_important,shared_interests_important,attractive_partner,sincere_partner,intelligence_partner,funny_partner,ambition_partner,shared_interests_partner,interests_correlate,expected_happy_with_sd_people,expected_num_interested_in_me,like,guess_prob_liked,met,match
0,0,0,female,21.0,27.0,Asian/PacificIslander/Asian-American,European/Caucasian-American,2.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,6.0,8.0,8.0,8.0,8.0,6.0,15.0,20.0,20.0,15.0,15.0,15.0,6.0,9.0,7.0,7.0,6.0,5.0,0.14,3.0,2.0,7.0,6.0,0.0,0
1,1,0,female,21.0,22.0,Asian/PacificIslander/Asian-American,European/Caucasian-American,2.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,7.0,8.0,10.0,7.0,7.0,5.0,15.0,20.0,20.0,15.0,15.0,15.0,7.0,8.0,7.0,8.0,5.0,6.0,0.54,3.0,2.0,7.0,5.0,1.0,0
2,2,1,female,21.0,22.0,Asian/PacificIslander/Asian-American,Asian/PacificIslander/Asian-American,2.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,10.0,10.0,10.0,10.0,10.0,10.0,15.0,20.0,20.0,15.0,15.0,15.0,5.0,8.0,9.0,8.0,5.0,7.0,0.16,3.0,2.0,7.0,NaN,1.0,1
3,3,0,female,21.0,23.0,Asian/PacificIslander/Asian-American,European/Caucasian-American,2.0,4.0,30.0,5.0,15.0,40.0,5.0,5.0,7.0,8.0,9.0,8.0,9.0,8.0,15.0,20.0,20.0,15.0,15.0,15.0,7.0,6.0,8.0,7.0,6.0,8.0,0.61,3.0,2.0,7.0,6.0,0.0,1
4,4,0,female,21.0,24.0,Asian/PacificIslander/Asian-American,Latino/HispanicAmerican,2.0,4.0,30.0,10.0,20.0,10.0,10.0,20.0,8.0,7.0,9.0,6.0,9.0,7.0,15.0,20.0,20.0,15.0,15.0,15.0,5.0,6.0,7.0,7.0,6.0,6.0,0.21,3.0,2.0,6.0,6.0,0.0,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Data columns (total 40 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     8378 non-null   int64  
 1   has_null                       8378 non-null   int64  
 2   gender                         8378 non-null   object 
 3   age                            8283 non-null   float64
 4   age_o                          8274 non-null   float64
 5   race                           8315 non-null   object 
 6   race_o                         8305 non-null   object 
 7   importance_same_race           8299 non-null   float64
 8   importance_same_religion       8299 non-null   float64
 9   pref_o_attractive              8289 non-null   float64
 10  pref_o_sincere                 8289 non-null   float64
 11  pref_o_intelligence            8289 non-null   float64
 12  pref_o_funny                   8280 non-null   f

- total : 8378 / 39cols
- has_null : null값 여부
- gender : 응답자 성별
- age/age_o : 본인나이/상대방 나이
- race/race_o : 본인 인종/상대방 인종
- importance_same_race/religion : 인종/종교를 중요시 여기는지 평가점수
- perf_o_xxx : 상대방이 xxx 항목을 얼마나 중요시하는지에 대한 점수
- xxx_o : 상대방이 본인의 xxx 항목을 평가한 점수
- xxx_important : xxx 항목에 대해 본인이 얼마나 중요하게 생각하는지에 대한 점수
- xxx_partner : 본인이 상대방에 대한 xxx 항목 평가
- interests_correlate : 관심사 연관도
- expectd_happy_with_sd_people : 스피드 데이팅을 통해 만난 사람과 함께 할 때 , 얼마나 좋을지에 대한 기대치
- expected_num_interested_in_me : 얼마나 많은 사람이 나에게 관심을 보일지에 대한 기대치
- like : 파트너를 좋아하는지
- guss_prob_liked : 파트너가 나를 마음에 들어했을지에 대한 예상
- met : 이전에 만난 적이 있는지
- match : 종속변수(target)

In [ ]:
round(df.describe().T, 2)

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,8378.0,4188.50,2418.66,0.00,2094.25,4188.50,6282.75,8377.00
has_null,8378.0,0.87,0.33,0.00,1.00,1.00,1.00,1.00
age,8283.0,26.36,3.57,18.00,24.00,26.00,28.00,55.00
age_o,8274.0,26.36,3.56,18.00,24.00,26.00,28.00,55.00
importance_same_race,8299.0,3.78,2.85,0.00,1.00,3.00,6.00,10.00
importance_same_religion,8299.0,3.65,2.81,1.00,1.00,3.00,6.00,10.00
pref_o_attractive,8289.0,22.50,12.57,0.00,15.00,20.00,25.00,100.00
pref_o_sincere,8289.0,17.40,7.04,0.00,15.00,18.37,20.00,60.00
pref_o_intelligence,8289.0,20.27,6.78,0.00,17.39,20.00,23.81,50.00
pref_o_funny,8280.0,17.46,6.09,0.00,15.00,18.00,20.00,50.00


- 최대값이 특이한 부분을 확인
- 본인 및 상대방을 평가하는 변수(xxx_o, xxx_partner)는 최대값이 10으로 일정해보인다.
- 중요도와 관련된 변수(pref_o_xxx, xxx_important)는 최대값이 100부터 60, 50...
- 평가 관련된 변수는 대체적으로 0-10으로 개별 항목에 적용
- 중요도 관련 변수는 최대값 100점을 각 항목에 분배하여 적용


In [ ]:
# 항목별 결측치 비율
df.isnull().mean() * 100

Unnamed: 0                        0.000000
has_null                          0.000000
gender                            0.000000
age                               1.133922
age_o                             1.241346
race                              0.751969
race_o                            0.871330
importance_same_race              0.942946
importance_same_religion          0.942946
pref_o_attractive                 1.062306
pref_o_sincere                    1.062306
pref_o_intelligence               1.062306
pref_o_funny                      1.169730
pref_o_ambitious                  1.277154
pref_o_shared_interests           1.539747
attractive_o                      2.530437
sincere_o                         3.425639
intelligence_o                    3.652423
funny_o                           4.296968
ambitous_o                        8.617809
shared_interests_o               12.843161
attractive_important              0.942946
sincere_important                 0.942946
intellicenc

**결측치 처리**
- 일반적으로 모든 변수의 결측치가 5% 미만
- 숫자데이터는 범위를 벗어나거나 일반적으로 나올법하지 않은 숫자는 -99로 대체
- 중요도와 관련된 변수들은 결측치를 제거
- 중요도 * 점수 계산해 100점 만점의 값으로 처리
- dropna() : 결측치 제거

## 1-1. 데이터 탐색 및 전처리

In [ ]:
df.columns

Index(['Unnamed: 0', 'has_null', 'gender', 'age', 'age_o', 'race', 'race_o',
       'importance_same_race', 'importance_same_religion', 'pref_o_attractive',
       'pref_o_sincere', 'pref_o_intelligence', 'pref_o_funny',
       'pref_o_ambitious', 'pref_o_shared_interests', 'attractive_o',
       'sincere_o', 'intelligence_o', 'funny_o', 'ambitous_o',
       'shared_interests_o', 'attractive_important', 'sincere_important',
       'intellicence_important', 'funny_important', 'ambtition_important',
       'shared_interests_important', 'attractive_partner', 'sincere_partner',
       'intelligence_partner', 'funny_partner', 'ambition_partner',
       'shared_interests_partner', 'interests_correlate',
       'expected_happy_with_sd_people', 'expected_num_interested_in_me',
       'like', 'guess_prob_liked', 'met', 'match'],
      dtype='object')

## 1-2. 일부 feature의 결측치 제거
- dropna(subset = [])
- 중요도와 관련된 변수들의 결측치 제거

In [ ]:
df = df.dropna(subset = ['pref_o_attractive',
       'pref_o_sincere', 'pref_o_intelligence', 'pref_o_funny',
       'pref_o_ambitious', 'pref_o_shared_interests', 'attractive_important', 'sincere_important',
       'intellicence_important', 'funny_important', 'ambtition_important',
       'shared_interests_important'
                         ])

- 나머지 변수에 존재하는 결측치는 -99로 대체

In [ ]:
df = df.fillna(-99)

## 1-3. 피처 엔지니어링

- 파생변수를 생성하거나 변수들을 이용하여 계산을 수행하는 것처럼 각 피처들의 값을 조정

In [ ]:
# age를 처리할 목적의 함수를 구현
def age_gap(x):
  if x['age'] == -99:
    return -99
  elif x['age_o'] == -99:
    return -99
  # 연상 또는 연하
  elif x['gender'] == 'female':
    return x['age_o'] - x['age']
  else:
    return x['age'] - x['age_o']

- 남녀 중 한명이라도 -99이면 -99를 반환
- 남자가 연상이면 플러스, 여자가 연상이면 마이너스값 반환

In [ ]:
df['age_gap'] = df.apply(age_gap, axis = 1)    # apply.() : 반복적용
# age_gap 변수의 절대값을 이용하여 순수 나이차를 담는다.
df['age_gap_abs'] = abs(df['age_gap'])

- 본인과 상대방의 인종이 같으면 1, 다르면 -1로 처리, 결측치는 -99로 처리

In [ ]:
# 인종 처리 목적의 함수를 구현
def same_race(x):
  if x['race'] == -99:
    return -99
  elif x['race_o'] == -99:
    return -99
  elif x['race'] == x['race_o']:
    return 1
  else:
    return -1


In [ ]:
df['same_race'] = df.apply(same_race, axis = 1)

In [ ]:
def same_race_point(x):
  if x['same_race'] == 99:
    return -99
  else :
    return x['same_race'] * x['importance_same_race']

In [ ]:
df['same_race_point'] = df.apply(same_race_point, axis = 1)

- 평가항목과 중요도를 곱해서 파생변수 생성

In [ ]:
def rating(data, importance, score):
  if data[importance] == -99:
    return -99
  elif data[score] == -99:
    return -99
  else:
    return data[importance] * data[score]

In [ ]:
df.columns

Index(['Unnamed: 0', 'has_null', 'gender', 'age', 'age_o', 'race', 'race_o',
       'importance_same_race', 'importance_same_religion', 'pref_o_attractive',
       'pref_o_sincere', 'pref_o_intelligence', 'pref_o_funny',
       'pref_o_ambitious', 'pref_o_shared_interests', 'attractive_o',
       'sincere_o', 'intelligence_o', 'funny_o', 'ambitous_o',
       'shared_interests_o', 'attractive_important', 'sincere_important',
       'intellicence_important', 'funny_important', 'ambtition_important',
       'shared_interests_important', 'attractive_partner', 'sincere_partner',
       'intelligence_partner', 'funny_partner', 'ambition_partner',
       'shared_interests_partner', 'interests_correlate',
       'expected_happy_with_sd_people', 'expected_num_interested_in_me',
       'like', 'guess_prob_liked', 'met', 'match', 'age_gap', 'age_gap_abs',
       'same_race', 'same_race_point'],
      dtype='object')

In [ ]:
df.columns[8:14]

Index(['importance_same_religion', 'pref_o_attractive', 'pref_o_sincere',
       'pref_o_intelligence', 'pref_o_funny', 'pref_o_ambitious'],
      dtype='object')

In [ ]:
# 각 항목에 대한 변수들을 리스트로 생성

## 상대방의 중요도
partner_imp = df.columns[8:14]

## 나에 대한 상대방의 평가
partner_rate_me = df.columns[14:20]

## 나의 중요도
my_imp = df.columns[20:26]

## 상대방에 대한 나의 평가
my_rate_partner = df.columns[26:32]

- 각 항목(6가지)에 대한 파생변수 생성
- 항목별 점수(중요도 * 평가점수)를 담는다.
- 'attractive_p', 'sincere_p', 'intelligence_p', 'funny_p', 'ambitous_p', 'shared_interests_p'

In [ ]:
# 상대방 관련 점수를 담을 변수 리스트
new_label_partner = ['attractive_p', 'sincere_p', 'intelligence_p', 'funny_p', 'ambitous_p', 'shared_interests_p']

# 나 관련 점수를 담을 리스트
new_label_me = ['attractive_m', 'sincere_m', 'intelligence_m', 'funny_m', 'ambitous_m', 'shared_interests_m']

In [ ]:
# 상대방 관련 변수 생성
for i, j, k in zip(new_label_partner, partner_imp, partner_rate_me):
  df[i] = df.apply(lambda x : rating(x, j, k), axis = 1)

In [ ]:
# 나 관련 변수 생성
for i, j, k in zip(new_label_me, my_imp, my_rate_partner):
  df[i] = df.apply(lambda x : rating(x, j, k), axis = 1)

- object 타입의 변수들 gender, race, race_o 정형화

In [ ]:
df = pd.get_dummies(df, columns = ['gender', 'race', 'race_o'], drop_first = True)   # drop_first : 첫번쨰 컬럼 삭제

# 2. 모델링과 평가

In [ ]:
## 2-1. 훈련세트와 테스트세트 분리

In [ ]:
from sklearn.model_selection import train_test_split
# x : feature
# y : target
x_train, x_test, y_train, y_test = train_test_split(df.drop('match', axis = 1), df['match'],
                                                    test_size = 0.2, random_state = 100)

## 2-1. XGBoost 모델 구현

In [ ]:
import xgboost as xgb

In [ ]:
model = xgb.XGBClassifier(n_estimators = 500, max_depth = 5, random_state = 100)
model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=100, ...)

In [ ]:
pred = model.predict(x_test)

In [ ]:
print(model.score(x_train, y_train))
print(model.score(x_test, y_test))

1.0
0.8763837638376384


In [ ]:
# 평가지표
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

accuracy_score(y_test, pred)

0.8763837638376384

In [ ]:
print(confusion_matrix(y_test, pred))

[[1304   61]
 [ 140  121]]


- 1종오류 : 우측 상단, 실제는 0인데 1로 예측
- 2종오류 : 좌측 하단, 실제는 1인데 0으로 예측

In [ ]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.90      0.96      0.93      1365
           1       0.66      0.46      0.55       261

    accuracy                           0.88      1626
   macro avg       0.78      0.71      0.74      1626
weighted avg       0.86      0.88      0.87      1626

